In [3]:
!pip install -q "qdrant-client[fastembed]>=1.14.2"

In [20]:
from fastembed import TextEmbedding
import numpy as np

In [125]:
from qdrant_client import QdrantClient,models

## Q1

In [55]:
query ='I just discovered the course. Can I join now?'

In [56]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
embeddings = list(model.embed(query))

In [57]:
q1_array=np.array(embeddings)

In [59]:
Q1_min = np.min(q1_array)

In [62]:
print("Q1=",Q1_min)

Q1= -0.11726373885183883


## Q2

In [64]:
doc ='Can I still join the course after the start date?'
embeddingsQ2 = list(model.embed(doc))
q2_array=np.array(embeddingsQ2)

In [70]:
Q2_answer=np.dot(q2_array,q1_array.T)

In [71]:
print("Q2=",Q2_answer)

Q2= [[0.90085289]]


## Q3

In [72]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [87]:
max_cos=0
max_index=0
index=0
for doc in documents:
    doc_embedding = list(model.embed(doc['text']))
    doc_array=np.array(doc_embedding)
    cos_q=np.dot(q1_array,doc_array.T)
    print(index,":",cos_q)
    if cos_q>max_cos:
        max_cos=cos_q
        max_index=index
    index=index+1

0 : [[0.76296845]]
1 : [[0.81823782]]
2 : [[0.80853974]]
3 : [[0.71330788]]
4 : [[0.73044992]]


In [89]:
print("Q3=",max_index,":",max_cos)

Q3= 1 : [[0.81823782]]


## Q4

In [91]:
max_cos=0
max_index=0
index=0
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    doc_embedding = list(model.embed(full_text))
    doc_array=np.array(doc_embedding)
    cos_q=np.dot(q1_array,doc_array.T)
    print(index,":",cos_q)
    if cos_q>max_cos:
        max_cos=cos_q
        max_index=index
    index=index+1

0 : [[0.85145432]]
1 : [[0.8436594]]
2 : [[0.84082872]]
3 : [[0.77551577]]
4 : [[0.80860079]]


In [92]:
print("Q4=",max_cos,":",max_index)

Q4= [[0.85145432]] : 0


## Q5

In [112]:
small_name=""
small_size=8192
for model in TextEmbedding.list_supported_models():
    ## print(int(model["dim"]))
    if small_size>int(model["dim"]):
        small_size=int(model["dim"])
        small_name=model["model"]

In [114]:
print ("Q5=",small_name,":",small_size)

Q5= BAAI/bge-small-en : 384


## Q6

In [116]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [128]:
model_name="BAAI/bge-small-en"
model = TextEmbedding(model_name)


In [129]:
client = QdrantClient("http://localhost:6333")

In [130]:
collection_name="llm-zoomcamp"
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    )
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `llm-zoomcamp` already exists!"},"time":0.000115388}'

In [ ]:
points=[]
id=0
for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector= models.Document(text=text,model=model_name),
        payload={
            "text": doc["text"],
            "section": doc["section"],
            "course": doc["course"]

        }
    )
    points.append(point)
    id=id+1

In [134]:
client.upsert(
    points=points,
    collection_name=collection_name
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [138]:
def search(query, limit=1,model_name=""):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_name 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [139]:
query ='I just discovered the course. Can I join now?'

In [140]:
result = search(query,1,model_name)

In [141]:
result

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])